# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [20]:
import numpy as np
import pandas as pd

In [21]:
# your code here
tweets = pd.read_csv('ds.csv', encoding='latin-1', header = None)
tweets.columns = ['target','id','date','flag','user','text']
tweets_sample = tweets.sample(20000)


In [22]:
tweets_sample

,target,id,date,flag,user,text
640254,0,2234934284,Fri Jun 19 00:03:45 PDT 2009,NO_QUERY,SamanthaJudd,"Im crushed, How could i have been so stupid?...."
894764,4,1692412092,Sun May 03 19:56:29 PDT 2009,NO_QUERY,Su_gey,recovery day. hahahahaha! so much fun last ni...
236056,0,1979902417,Sun May 31 04:57:52 PDT 2009,NO_QUERY,Bethlaaa,@AnthonyW90 oh my god...im freaking out that i...
864486,4,1677294866,Sat May 02 01:21:16 PDT 2009,NO_QUERY,nathanryder,@jensenwilder How long you planning on stayin...
1302490,4,2008444024,Tue Jun 02 14:08:25 PDT 2009,NO_QUERY,CassyMichelle,27 years old &amp; just had my 1st experience ...
...,...,...,...,...,...,...
935855,4,1792845522,Thu May 14 00:26:43 PDT 2009,NO_QUERY,DaisyHaryani,saw NKOTB performance on E! last night... So h...
1416542,4,2057423667,Sat Jun 06 12:58:48 PDT 2009,NO_QUERY,Alphamoongirl,@InThisCity Since last July 31st! It's KM.com'...
995782,4,1835772037,Mon May 18 06:43:06 PDT 2009,NO_QUERY,yaseminx3,@TheRealJordin saay hi to ryan from mee aka ya...
1020259,4,1882339366,Fri May 22 06:41:34 PDT 2009,NO_QUERY,UrbanAngelM,@antonyadelaar he's our personal slave. servin...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [23]:
import re 
import nltk
from nltk.corpus import stopwords #stopwords library
from nltk.stem import WordNetLemmatizer #lemmatizing library
from nltk.tokenize import word_tokenize #tokenizing library
from nltk.stem import PorterStemmer 

In [24]:
# your code here
def clean_up(s):
    output = re.sub(r'http\S+', '', s).lower()
    return re.sub('[^A-Za-z]+', ' ', output).lower().strip()

def tokenize(s):
    tok = word_tokenize(s.lower())
    return tok

def stem_and_lemmatize(l):
    stem = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    new = [lemmatizer.lemmatize(stem.stem(i)) for i in l]
    return new

def remove_stopwords(x):
    stop_words = stopwords.words('english')
    text = [t for t in x if t not in stop_words]
    return text



In [25]:

tweets_sample['text_processed'] = tweets_sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

tweets_sample

,target,id,date,flag,user,text,text_processed
640254,0,2234934284,Fri Jun 19 00:03:45 PDT 2009,NO_QUERY,SamanthaJudd,"Im crushed, How could i have been so stupid?....","[im, crush, could, stupid, nth]"
894764,4,1692412092,Sun May 03 19:56:29 PDT 2009,NO_QUERY,Su_gey,recovery day. hahahahaha! so much fun last ni...,"[recoveri, day, hahahahaha, much, fun, last, n..."
236056,0,1979902417,Sun May 31 04:57:52 PDT 2009,NO_QUERY,Bethlaaa,@AnthonyW90 oh my god...im freaking out that i...,"[anthonyw, oh, god, im, freak, skin, cancer]"
864486,4,1677294866,Sat May 02 01:21:16 PDT 2009,NO_QUERY,nathanryder,@jensenwilder How long you planning on stayin...,"[jensenwild, long, plan, stay, town]"
1302490,4,2008444024,Tue Jun 02 14:08:25 PDT 2009,NO_QUERY,CassyMichelle,27 years old &amp; just had my 1st experience ...,"[year, old, amp, st, experi, w, ga, denist, of..."
...,...,...,...,...,...,...,...
935855,4,1792845522,Thu May 14 00:26:43 PDT 2009,NO_QUERY,DaisyHaryani,saw NKOTB performance on E! last night... So h...,"[saw, nkotb, perform, e, last, night, happi, c..."
1416542,4,2057423667,Sat Jun 06 12:58:48 PDT 2009,NO_QUERY,Alphamoongirl,@InThisCity Since last July 31st! It's KM.com'...,"[inthisc, sinc, last, juli, st, km, com, versi..."
995782,4,1835772037,Mon May 18 06:43:06 PDT 2009,NO_QUERY,yaseminx3,@TheRealJordin saay hi to ryan from mee aka ya...,"[therealjordin, saay, hi, ryan, mee, aka, yase..."
1020259,4,1882339366,Fri May 22 06:41:34 PDT 2009,NO_QUERY,UrbanAngelM,@antonyadelaar he's our personal slave. servin...,"[antonyadelaar, person, slave, serv, chic, cof..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [26]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


In [27]:
# your code here
import itertools
import collections

freq_dist = nltk.FreqDist()

for tweet in tweets_sample['text_processed']:
    for word in tweet:
        freq_dist[word] += 1
        
x = list(itertools.islice(freq_dist.items(), 0, 5000))


In [28]:
x

[('im', 641),
 ('crush', 5),
 ('could', 290),
 ('stupid', 124),
 ('nth', 1),
 ('recoveri', 4),
 ('day', 1337),
 ('hahahahaha', 5),
 ('much', 424),
 ('fun', 350),
 ('last', 489),
 ('night', 605),
 ('consequ', 1),
 ('today', 838),
 ('anthonyw', 1),
 ('oh', 491),
 ('god', 131),
 ('freak', 36),
 ('skin', 8),
 ('cancer', 8),
 ('jensenwild', 1),
 ('long', 218),
 ('plan', 119),
 ('stay', 118),
 ('town', 51),
 ('year', 230),
 ('old', 177),
 ('amp', 638),
 ('st', 63),
 ('experi', 15),
 ('w', 154),
 ('ga', 17),
 ('denist', 1),
 ('offic', 62),
 ('yike', 10),
 ('meerasapra', 2),
 ('love', 1062),
 ('comic', 12),
 ('improv', 6),
 ('blog', 79),
 ('look', 509),
 ('layout', 1),
 ('etc', 20),
 ('though', 278),
 ('kinda', 65),
 ('crappi', 22),
 ('right', 391),
 ('good', 1180),
 ('morn', 448),
 ('bless', 40),
 ('yay', 171),
 ('saturday', 85),
 ('stereo', 4),
 ('die', 112),
 ('someth', 194),
 ('car', 125),
 ('got', 897),
 ('ta', 127),
 ('rip', 34),
 ('dash', 2),
 ('messag', 46),
 ('tweep', 21),
 ('ask', 93

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [49]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in x:
        features[w] = (w in words)
    
    return features
        
feature_sets = [(find_features(tweet), target) for (tweet, target) in list(zip(x['text_processed'], x['target']))]

TypeError: list indices must be integers or slices, not str

In [50]:
feature_sets[0]

({('im', 641): False,
  ('crush', 5): False,
  ('could', 290): False,
  ('stupid', 124): False,
  ('nth', 1): False,
  ('recoveri', 4): False,
  ('day', 1337): False,
  ('hahahahaha', 5): False,
  ('much', 424): False,
  ('fun', 350): False,
  ('last', 489): False,
  ('night', 605): False,
  ('consequ', 1): False,
  ('today', 838): False,
  ('anthonyw', 1): False,
  ('oh', 491): False,
  ('god', 131): False,
  ('freak', 36): False,
  ('skin', 8): False,
  ('cancer', 8): False,
  ('jensenwild', 1): False,
  ('long', 218): False,
  ('plan', 119): False,
  ('stay', 118): False,
  ('town', 51): False,
  ('year', 230): False,
  ('old', 177): False,
  ('amp', 638): False,
  ('st', 63): False,
  ('experi', 15): False,
  ('w', 154): False,
  ('ga', 17): False,
  ('denist', 1): False,
  ('offic', 62): False,
  ('yike', 10): False,
  ('meerasapra', 2): False,
  ('love', 1062): False,
  ('comic', 12): False,
  ('improv', 6): False,
  ('blog', 79): False,
  ('look', 509): False,
  ('layout', 1): F

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [51]:
# your code here
train_set, test_set = feature_sets[:10000], feature_sets[10000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [52]:
# your code here
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features()

0.491
Most Informative Features
            ('aaaah', 3) = False               0 : 4      =      1.0 : 1.0
             ('aaah', 4) = False               0 : 4      =      1.0 : 1.0
           ('aaawww', 4) = False               0 : 4      =      1.0 : 1.0
      ('aaawwwwwwww', 1) = False               0 : 4      =      1.0 : 1.0
           ('aachen', 1) = False               0 : 4      =      1.0 : 1.0
              ('aah', 4) = False               0 : 4      =      1.0 : 1.0
             ('aahh', 1) = False               0 : 4      =      1.0 : 1.0
            ('aaron', 6) = False               0 : 4      =      1.0 : 1.0
         ('aaronful', 1) = False               0 : 4      =      1.0 : 1.0
          ('abandon', 2) = False               0 : 4      =      1.0 : 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here